In [ ]:
# !pip install transformers==4.46.3 accelerate peft

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [18]:
import json
import peft
from datasets import load_dataset
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch

In [5]:
path_ = "/content/gdrive/MyDrive/TelegramData/prepared_data"

stack_ = lambda x: os.path.join(path_, x)
train_dataset = load_dataset("json", data_files=stack_("train.json"), split="train")
val_dataset = load_dataset("json", data_files=stack_("val.json"), split="train")
test_dataset = load_dataset("json", data_files=stack_("test.json"), split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
model_name = 'Qwen/Qwen2.5-0.5B'
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

MAX_POST_LEN = 128
MAX_TOTAL_LEN = 192

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def preprocess(ex):
    prompt = ex["input_text"] + "\nКомментарий: "
    target = ex["output_text"] + tokenizer.eos_token

    prompt_ids = tokenizer(
        prompt,
        add_special_tokens=False,
        truncation=True,
        max_length=MAX_POST_LEN,
    )["input_ids"]

    target_ids = tokenizer(
        target,
        add_special_tokens=False,
        truncation=False,
    )["input_ids"]

    input_ids = prompt_ids + target_ids
    labels = [-100] * len(prompt_ids) + target_ids

    # финальное ограничение длины
    input_ids = input_ids[:MAX_TOTAL_LEN]
    labels = labels[:MAX_TOTAL_LEN]

    # ручной паддинг
    pad_len = MAX_TOTAL_LEN - len(input_ids)
    if pad_len > 0:
        input_ids += [tokenizer.pad_token_id] * pad_len
        labels += [-100] * pad_len

    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": [1] * (MAX_TOTAL_LEN - pad_len) + [0] * pad_len
    }

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/290 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [6]:
save_dir = "/content/gdrive/MyDrive/diploma/lora_weights"

In [8]:
train_dataset = train_dataset.map(preprocess)
val_dataset = val_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

Map:   0%|          | 0/8412 [00:00<?, ? examples/s]

Map:   0%|          | 0/1052 [00:00<?, ? examples/s]

Map:   0%|          | 0/1052 [00:00<?, ? examples/s]

In [9]:
from peft import LoraConfig, get_peft_model, TaskType

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 540,672 || all params: 494,573,440 || trainable%: 0.1093


In [10]:
training_args = TrainingArguments(
    output_dir=os.path.join(save_dir, model_name),
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer,
)


In [11]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Epoch,Training Loss,Validation Loss
1,3.368869,3.392186
2,3.299012,3.366580


TrainOutput(global_step=2104, training_loss=3.376128338135694, metrics={'train_runtime': 3508.5878, 'train_samples_per_second': 4.795, 'train_steps_per_second': 0.6, 'total_flos': 6946990965522432.0, 'train_loss': 3.376128338135694, 'epoch': 2.0})

In [12]:
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('/content/gdrive/MyDrive/diploma/lora_weights/tokenizer_config.json',
 '/content/gdrive/MyDrive/diploma/lora_weights/chat_template.jinja',
 '/content/gdrive/MyDrive/diploma/lora_weights/tokenizer.json')

In [13]:
logs_part1 = trainer.state.log_history.copy()

In [14]:
post_text = """В октябре световой день в Москве сократится на 2 часа 15 минут.

По данным Московского планетария, в начале месяца солнце будет светить 11 часов 34 минуты, а к концу — только 9 часов 19 минут."""
post_text = test_dataset["input_text"][225]
# cluster_id = 1  # тип реакции
prompt = f"{post_text} \nКомментарий:"

In [24]:
def generation(model):
  random.seed(42)
  for _ in range(3):
    post_text = test_dataset["input_text"][random.randint(0, 1000)]

    prompt = f"{post_text} \nКомментарий:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=210,
        top_p=0.9,
        no_repeat_ngram_size=3,
        repetition_penalty=1.2,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
    print(generated_text)
    print()

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [22]:
import random
generation(model)

Пост: 🏙️ В районе «Лахта-центра» могут вырасти новые небоскрёбы.

Городская комиссия уже одобрила предложение прописать в правилах землепользования и застройки минимальную высоту зданий — не ниже 300 метров. Это означает, что рядом с башней «Газпрома» потенциально появится целый квартал высоток.

Компания «Синергия», которая готовит документы для строительства в Лахте, заявляла, что новые башни будут полностью заселены только после появления метро рядом. По планам, его открытие вряд ли стоит ждать раньше 2035 года. 
Комментарий:Нет, это просто миф. Скорее всего, это будет пустая площадь<|endoftext|>

Пост: За что убил?
За то, что подошёл ко мне, не объяснив причину!

Первичный допрос задержанного в лесополосе 19-летнего Руслана Тимофеева — парень признаётся в убийстве ножом в шею 24-летнего оперативника и объясняет свои причины этого поступка.

По словам Руслана, он готовил план, "чтобы воевать с теми, кто придёт на его поиски" — планировал напасть и убить также и разыскивающих его сил

Видим, что модель снова отвечает "слишком общо" => необходимо еще несколько эпох

In [23]:
training_args2 = TrainingArguments(
    output_dir=os.path.join(save_dir, model_name),
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=False,
)

trainer = Trainer(
    model=model,
    args=training_args2,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,3.242156,3.318689
2,3.204578,3.312348


TrainOutput(global_step=2104, training_loss=3.227475431028881, metrics={'train_runtime': 3512.266, 'train_samples_per_second': 4.79, 'train_steps_per_second': 0.599, 'total_flos': 6946990965522432.0, 'train_loss': 3.227475431028881, 'epoch': 2.0})

In [25]:
logs_part2 = logs_part1 + trainer.state.log_history

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('/content/gdrive/MyDrive/diploma/lora_weights/tokenizer_config.json',
 '/content/gdrive/MyDrive/diploma/lora_weights/chat_template.jinja',
 '/content/gdrive/MyDrive/diploma/lora_weights/tokenizer.json')

In [26]:
generation(model)

Пост: 🏙️ В районе «Лахта-центра» могут вырасти новые небоскрёбы.

Городская комиссия уже одобрила предложение прописать в правилах землепользования и застройки минимальную высоту зданий — не ниже 300 метров. Это означает, что рядом с башней «Газпрома» потенциально появится целый квартал высоток.

Компания «Синергия», которая готовит документы для строительства в Лахте, заявляла, что новые башни будут полностью заселены только после появления метро рядом. По планам, его открытие вряд ли стоит ждать раньше 2035 года. 
Комментарий:Нет, это не так. Странно, но я думаю, что это просто миф<|endoftext|>

Пост: За что убил?
За то, что подошёл ко мне, не объяснив причину!

Первичный допрос задержанного в лесополосе 19-летнего Руслана Тимофеева — парень признаётся в убийстве ножом в шею 24-летнего оперативника и объясняет свои причины этого поступка.

По словам Руслана, он готовил план, "чтобы воевать с теми, кто придёт на его поиски" — планировал напасть и убить также и разыскивающих его силови

Несмотря на падение loss'а и то, что модель стала отвечать конкретнее, ее ответы не соответствуют смыслу постов.

Продолжим обучение

In [27]:
training_args3 = TrainingArguments(
    output_dir=os.path.join(save_dir, model_name),
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=False,
)

trainer = Trainer(
    model=model,
    args=training_args3,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,3.176018,3.307761
2,3.147649,3.308055


TrainOutput(global_step=2104, training_loss=3.1571473241305625, metrics={'train_runtime': 3522.1883, 'train_samples_per_second': 4.777, 'train_steps_per_second': 0.597, 'total_flos': 6946990965522432.0, 'train_loss': 3.1571473241305625, 'epoch': 2.0})

In [28]:
generation(model)

Пост: 🏙️ В районе «Лахта-центра» могут вырасти новые небоскрёбы.

Городская комиссия уже одобрила предложение прописать в правилах землепользования и застройки минимальную высоту зданий — не ниже 300 метров. Это означает, что рядом с башней «Газпрома» потенциально появится целый квартал высоток.

Компания «Синергия», которая готовит документы для строительства в Лахте, заявляла, что новые башни будут полностью заселены только после появления метро рядом. По планам, его открытие вряд ли стоит ждать раньше 2035 года. 
Комментарий:Нет, это не так. Скорее всего, они будут на уровне 10 этажей<|endoftext|>

Пост: За что убил?
За то, что подошёл ко мне, не объяснив причину!

Первичный допрос задержанного в лесополосе 19-летнего Руслана Тимофеева — парень признаётся в убийстве ножом в шею 24-летнего оперативника и объясняет свои причины этого поступка.

По словам Руслана, он готовил план, "чтобы воевать с теми, кто придёт на его поиски" — планировал напасть и убить также и разыскивающих его си

Видим, что качество улучшилось, ответы стали ближе по смыслу к посту. Тем не менее - еще не ухватывают достаточно сильно смысл

In [29]:
logs_part3 = logs_part2 + trainer.state.log_history

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('/content/gdrive/MyDrive/diploma/lora_weights/tokenizer_config.json',
 '/content/gdrive/MyDrive/diploma/lora_weights/chat_template.jinja',
 '/content/gdrive/MyDrive/diploma/lora_weights/tokenizer.json')

In [ ]:
training_args4 = TrainingArguments(
    output_dir=os.path.join(save_dir, model_name),
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=False,
)

trainer = Trainer(
    model=model,
    args=training_args4,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer,
)

trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
import matplotlib.pyplot as plt

logs = trainer.state.log_history
train_loss = [(x["epoch"], x["loss"]) for x in logs if "loss" in x]
val_loss = [(x["epoch"], x["eval_loss"]) for x in logs if "eval_loss" in x]

plt.plot(*zip(*train_loss), label="train")
plt.plot(*zip(*val_loss), label="val")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.savefig("loss_curve.png")
plt.show()